## Import Libraries & Load Datasets

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Read the CSV files into data frames
markcamp_clean = pd.read_csv('markcamp_clean.csv')  
onlineretail_clean = pd.read_csv('onlineretail_clean.csv')  
custseg_clean = pd.read_csv('custseg_clean.csv')  

## Ensure CustomerID column contains sales from actual customers

In [3]:
# Extract the list of IDs from markcamp_clean
ids = markcamp_clean['ID'].tolist()

# Replace 'CustomerID' in 'onlineretail_clean' with 'CustomerIDs' from the 'markcamp_clean' data set
onlineretail_clean['CustomerID'] = onlineretail_clean['CustomerID'].apply(lambda x: random.choice(ids))

In [4]:
# Rename ID column
markcamp_clean.rename(columns={'ID': 'CustomerID'}, inplace=True)

## Add 'CustomerID' column to 'custseg_clean' 

In [5]:
# Shuffle the list of IDs to ensure uniqueness
unique_ids = np.random.permutation(markcamp_clean['CustomerID'].tolist())

# Calculate the number of additional IDs needed
num_additional_ids = 51000 - len(unique_ids)

# Generate additional fake IDs
additional_fake_ids = np.arange(1, num_additional_ids + 1) + np.max(unique_ids)

# Combine unique IDs from markcamp_clean['ID'] with additional system-generated IDs
all_ids = np.concatenate((unique_ids, additional_fake_ids))

# Shuffle all_ids to ensure randomness
shuffled_ids = np.random.permutation(all_ids)

# Assign the shuffled CustomerIDs to a new column in custseg_clean
custseg_clean.insert(0, 'CustomerID', shuffled_ids)

## Merge 'markcamp_clean' & 'custseg_clean' datasets

In [6]:
# Verify individual unique IDs have been assigned to each customer
unique_customerseg_count = custseg_clean['CustomerID'].nunique()
print("Unique Customer Count:", unique_customerseg_count)


Unique Customer Count: 51000


In [7]:
unique_customer_count = onlineretail_clean['CustomerID'].nunique()
print("Unique Customer Count:", unique_customer_count)

Unique Customer Count: 2216


In [8]:
# Merge custseg_clean with markcamp_clean on 'CustomerID'
custseg_clean = pd.merge(custseg_clean, markcamp_clean, on='CustomerID', how='left')

In [9]:
custseg_clean.shape

(51000, 45)

## Fill profile data to ensure dataset is complete 

In [10]:
# Delete unwanted columns
columns_to_delete = [
    'MntFishProducts', 'MntFruits', 'MntGoldProds', 'MntMeatProducts', 
    'MntSweetProducts', 'MntWines', 'NumCatalogPurchases', 'NumDealsPurchases',
    'NumStorePurchases', 'NumWebPurchases']

# Drop the specified columns
custseg_clean.drop(columns=columns_to_delete, inplace=True)


In [11]:
# Define data ranges for each column
data_ranges = {
    'Education': ['2n Cycle', 'Basic', 'Graduation', 'Master', 'PhD'],
    'Income': range(1500, 750001),
    'Kidhome': range(3),
    'Marital_Status': ['Absurd', 'Alone', 'Divorced', 'Married', 'Single', 'Together', 'Widow', 'YOLO'],
    'NumWebVisitsMonth': range(21),  
    'Recency': range(100),
    'Response': [0, 1],
    'Teenhome': range(3),
    'Year_Birth': range(1940, 1997)
}

# Iterate over each column and populate with random data
for column, values in data_ranges.items():
    custseg_clean[column] = np.random.choice(values, size=len(custseg_clean))

## Create 'demographic_data' & 'geographic_data' datasets from 'custseg_clean' dataset

In [12]:
# Demographic Data Set
demographic_columns = ['CustomerID', 'first_name', 'last_name', 'title', 'gender', 'email',
                       'company_name', 'department', 'job_title','job_category', 'language', 
                       'university', 'linkedin_skill', 'Education', 'Income', 'Kidhome', 
                       'Marital_Status', 'Teenhome', 'Year_Birth']

demographic_data = custseg_clean[demographic_columns]

# Geographic Data Set
geographic_columns = ['CustomerID', 'city', 'country', 'country_code', 'region', 'latitude', 'longitude',
                      'phone', 'street_address', 'street_name', 'street_number','street_suffix', 
                      'time_zone', 'ip_address']

geographic_data = custseg_clean[geographic_columns]

# Printing first few rows of each set
print("Demographic Data Set:")
print(demographic_data.shape)

print("\nGeographic Data Set:")
print(geographic_data.shape)


Demographic Data Set:
(51000, 19)

Geographic Data Set:
(51000, 14)


## Validate Transformations

In [13]:
# Count null values in each column
null_counts_online = onlineretail_clean.isnull().sum()

# Display the null counts
print(null_counts_online)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
dtype: int64


In [14]:
# Count null values in each column
null_counts_demographic = demographic_data.isnull().sum()

# Display the null counts
print(null_counts_demographic)

CustomerID          0
first_name          0
last_name           0
title               0
gender              0
email               0
company_name        0
department          0
job_title           0
job_category      445
language            0
university          0
linkedin_skill      0
Education           0
Income              0
Kidhome             0
Marital_Status      0
Teenhome            0
Year_Birth          0
dtype: int64


In [15]:
# Count null values in each column
null_counts_geographic = geographic_data.isnull().sum()

# Display the null counts
print(null_counts_geographic)

CustomerID         0
city               0
country            0
country_code      34
region             0
latitude           0
longitude          0
phone              0
street_address     0
street_name        0
street_number      0
street_suffix      0
time_zone          0
ip_address         0
dtype: int64


In [16]:
unique_customer_count = onlineretail_clean['CustomerID'].nunique()
print("Number of unique CustomerIDs:", unique_customer_count)

Number of unique CustomerIDs: 2216


In [17]:
unique_customer_count = demographic_data['CustomerID'].nunique()
print("Number of unique CustomerIDs:", unique_customer_count)

Number of unique CustomerIDs: 51000


In [18]:
unique_customer_count = geographic_data['CustomerID'].nunique()
print("Number of unique CustomerIDs:", unique_customer_count)

Number of unique CustomerIDs: 51000


In [19]:
# # Write dataframes to csv files
# onlineretail_clean.to_csv('onlineretail_clean.csv', index=False)

# demographic_data.to_csv('custseg_clean.csv', index=False)

# geographic_data.to_csv('markcamp_clean.csv', index=False)